In [ ]:
pip install selenium


In [ ]:
# import packages
from selenium import webdriver
from time import sleep
from bs4 import BeautifulSoup
import re
import random
import unicodedata
import pandas as pd
import time

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [42]:
# output csv file name
CSV_FILENAME = 'gap_women_b.csv'
CAT_URL = "https://www.gap.com/browse/category.do?cid=34524&nav=hamnav%3AWomen%3ACategories%3ABras"

In [22]:
# Random sleep to mimic human behavior
def random_sleep(min_time=2, max_time=5):
    """
    minimum time to sleep set to 2
    max_time: maximum time to sleep set is 5
    """
    sleep_time = random.uniform(min_time, max_time)
    sleep(sleep_time)

In [23]:
import time
from random import randint
# to load more products:
def scroll(driver, timeout=30):
    scroll_pause_time = 2.5
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        else:
            last_height = new_height
            time.sleep(randint(1, 3))
# the timeout, pause_time, and time.sleep need to be adjusted according to case-by-case time consumption.

In [24]:
def get_soup(driver):
    return BeautifulSoup(driver.page_source, 'lxml')

In [25]:
def extract_product_url(product):
    try:
        href = product.find('a', href=True)['href']
        base_url = href.split('&')[0]
    except (AttributeError, TypeError):
        base_url = 'Not Available'

    return base_url

In [26]:
def extract_product_demo(soup):
    """
    Extracts the product demographic (men or women) from the product page.
    """
    try:
        product_demo_link = soup.find('a', class_='pdp-mfe-1lmagf7')
        product_demo = product_demo_link.get_text() if product_demo_link else 'Not available'
    except Exception as e:
        print(f"Error extracting product demographic: {e}")
        product_demo = 'error'

    return product_demo

In [27]:
def extract_product_type(soup):
    """
    Extracts the product category name (e.g., jeans, t-shirts, shirts).
    """
    try:
        product_type_links = soup.find_all('a', class_='pdp-mfe-1lmagf7')
        product_type = product_type_links[1].get_text() if len(product_type_links) > 1 else 'Not available'
    except Exception as e:
        print(f"Error extracting product type: {e}")
        product_type = 'error'

    return product_type

In [36]:
def extract_product_name(soup):
    try:
        name_element = soup.find('h1', class_='pdp-mfe-ch2lru')
        if name_element:
            product_name = name_element[0].text() 
        else:
            product_name = 'Not available'
    except Exception as e:
        print(f"An error occurred: {e}")
        product_name = 'Not available'
    
    return product_name

In [29]:
def extract_prices(soup):
    """
    Extracts the low price and max price from the product page,
    """
    try:
        low_price = soup.find('div', class_="product-price--pdp__highlight")
        if not low_price:
            low_price = soup.find('div', class_="pdp-pricing--highlight pdp-pricing__selected pdp-mfe-1jiw3bl")
        if not low_price:
            low_price = soup.find('div', class_=re.compile(r'product-price|pdp-pricing'))
        low_price = low_price.text.strip('$')
        low_price = re.sub(r'\([^()]*\)', '', low_price).strip()
        
        max_price = soup.find('div', class_="product-price--pdp__regular")
        if not max_price:
            max_price = soup.find('div', class_="product-price__strike pdp-mfe-vo1pn1")
        if max_price:
            max_price = max_price.text.strip('$')
        else:
            max_price = low_price  # Use the regular price if no separate max price is found
    except AttributeError: 
        low_price = 'Not available'
        max_price = 'Not available'
    
    return low_price, max_price

In [30]:
def extract_rating(soup):
    try:
        rating_element = soup.find('span', class_='pdp-mfe-3jhqep')
        if rating_element:
            rating_text = rating_element.text
            rating = rating_text.split(',')[-1].strip().split()[0]
        else:
            rating = 'Not available'
    except Exception as e:
        print(f"An error occurred: {e}")
        rating = 'Not available'
    
    return rating

In [31]:
def extract_ratings_count(soup):
    """
    Extracts the numeric rating count from the product page.
    Args:
    - soup: BeautifulSoup object of the product page.

    Returns:
    - rating_count: Integer representing the number of ratings, or 'Not available' if not found.
    """
    try:
        rating_element = soup.find('span', class_='pr-snippet-rating-count')
        if rating_element:
            rating_count = rating_element.text.split()[0]
        else:
            rating_count = 'Not available'
    except Exception as e:
        print(f"An error occurred: {e}")
        rating_count = 'Not available'
    
    return rating_count

In [32]:
def extract_color(soup):
    try:
        color_element = soup.find('span', class_='swatch-label_value')
        if color_element:
            color = color_element.text.strip()
        else:
            color = 'Not available'
    except Exception as e:
        print(f"An error occurred: {e}")
        color = 'Not available'
    
    return color

In [39]:
def extract_overview_and_details(soup):
    sections = soup.find_all('div', class_='pdp-mfe-1hb6eb9')
    if len(sections) > 1:
        product_details_section = sections[1]  
        details = {'overview': None, 'list_items': []}
        
        overview_element = product_details_section.find('p', class_='product-information-item__overview')
        if overview_element:
            details['overview'] = overview_element.text.strip()
        
        list_items = product_details_section.find_all('li', class_='product-information-item__list-item')
        details['list_items'] = [item.text.strip() for item in list_items]

        return details

In [40]:
def extract_fabric(soup):
    """
    Extracts fabric and care details from the third div.pdp-mfe-1hb6eb9 section.
    Args:
    - soup: BeautifulSoup object of the product page.

    Returns:
    - care_details: List of fabric care instructions.
    """
    sections = soup.find_all('div', class_='pdp-mfe-1hb6eb9')
    if len(sections) > 2:
        fabric_care_section = sections[2]
        care_items = fabric_care_section.find_all('li', class_='product-information-item__list-item')
        return [item.text.strip() for item in care_items]
    return []

In [34]:
import os

In [48]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time

def process_product_page(driver, url):
    driver.get(url)
    random_sleep()
    soup = get_soup(driver)
    product_info = {
        'Demographic': extract_product_demo(soup),
        'Category': extract_product_type(soup),
        'Product_Name': extract_product_name(soup),
        'Low_Price': extract_prices(soup)[0],
        'Max_Price': extract_prices(soup)[1],
        'Rating': extract_rating(soup),
        'Rating_Count': extract_ratings_count(soup),
        'Color': extract_color(soup),
        'Details': extract_overview_and_details(soup),
        'Fabrics': extract_fabric(soup)
    }
    return product_info

def initialize_driver():
    """Initializes the WebDriver."""
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(10)
    return driver

def main():
    driver = initialize_driver()
    driver.get(CAT_URL)
    scroll(driver)
    soup = get_soup(driver)
    product_info_elements = soup.find_all('div', class_='category-page-1wcebst')
    cat_url_list = [extract_product_url(product) for product in product_info_elements]

    product_data = []
    for url in cat_url_list:
        if url != 'Not available':
            try:
                product_info = process_product_page(driver, url)
                product_info['URL'] = url
                product_data.append(product_info)
                print(f"Product Collected {len(product_data)}")
            except Exception as e:
                print(f"Failed to process {url}: {e}")

    # Create DataFrame from collected data
    df = pd.DataFrame(product_data)
    df.to_csv(CSV_FILENAME, index=False)
    print(f"File {CSV_FILENAME} Created")
    driver.quit()

if __name__ == "__main__":
    main()


Product Collected 1
Product Collected 2
Product Collected 3
Product Collected 4
Product Collected 5
Product Collected 6
Product Collected 7
Product Collected 8
Failed to process Not Available: Message: invalid argument
  (Session info: chrome=124.0.6367.91)
Stacktrace:
	GetHandleVerifier [0x00007FF7A2F41502+60802]
	(No symbol) [0x00007FF7A2EBAC02]
	(No symbol) [0x00007FF7A2D77B13]
	(No symbol) [0x00007FF7A2D605E0]
	(No symbol) [0x00007FF7A2D5EC3D]
	(No symbol) [0x00007FF7A2D5F22C]
	(No symbol) [0x00007FF7A2D7A9F9]
	(No symbol) [0x00007FF7A2E0AB7E]
	(No symbol) [0x00007FF7A2DEAB7A]
	(No symbol) [0x00007FF7A2E0A224]
	(No symbol) [0x00007FF7A2DEA923]
	(No symbol) [0x00007FF7A2DB8FEC]
	(No symbol) [0x00007FF7A2DB9C21]
	GetHandleVerifier [0x00007FF7A324411D+3217821]
	GetHandleVerifier [0x00007FF7A32860B7+3488055]
	GetHandleVerifier [0x00007FF7A327F03F+3459263]
	GetHandleVerifier [0x00007FF7A2FFB846+823494]
	(No symbol) [0x00007FF7A2EC5F9F]
	(No symbol) [0x00007FF7A2EC0EC4]
	(No symbol) [0x0